In [1]:
%load_ext autoreload
import datetime
# import ete3
import itertools
import json
import logging
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from pathlib import Path
import os
import os.path
# import typing
# import re
# import xml.etree.ElementTree as ET
# import xml.dom.minidom
# import xmlschema

In [2]:
%autoreload
# from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
from maxes.analyze_xes import AnalyzeXes
# import maxes.analyze_sequence
# import maxes.serialization.serialize
# import maxes.graphs
import maxes.notebooks.utils
# import maxes.utils
from maxes.utils import action_logging

In [ ]:
maxes.notebooks.utils.init_notebook() # RUN ONLY ONCE

In [33]:
%autoreload
from maxes.data.manifest import manifest
from maxes.data.manifest_reader import process_manifest, process_manifest_item

# (show_progress_bar=True, skip_downloaded=False)
manifest_item = next(item for item in manifest if item['key'] == 'bpic2017')
print(manifest_item)

process_manifest_item(manifest_item, show_progress_bar=True, skip_downloaded=True)


# manifest
# [item['key'] for item in manifest]

{'key': 'bpic2017', 'title': 'BPI Challenge 2017', 'page_url': 'https://data.4tu.nl/articles/_/12696884/1', 'default_file_key': 'event_log', 'files': [{'key': 'event_log', 'url': 'https://data.4tu.nl/file/34c3f44b-3101-4ea9-8281-e38905c68b8d/f3aec4f7-d52c-4217-82f4-57d719a8298c', 'destination': 'BPI Challenge 2017.xes.gz'}], 'process': [{'step': 'gzip', 'source': 'BPI Challenge 2017.xes.gz', 'destination': 'BPI Challenge 2017.xes'}], 'result_files': [{'key': 'event_log', 'path': 'BPI Challenge 2017.xes'}]}


f3aec4f7-d52c-4217-82f4-57d719a8298c: 29.7MB [00:02, 11.3MB/s]                            


In [ ]:
from maxes.data.manifest import manifest

import os.path

result = []
for manifest_item in manifest:
    if 'result_files' not in manifest_item:
        print(f"Skipped entry {manifest_item['key']}")
        continue

    data_key = manifest_item['key']

    for result_files_item in manifest_item['result_files']:
        result_files_item_path = result_files_item['path']

        path = os.path.join('data', data_key, result_files_item_path)

        data_result_file_key = ""
        data_key = ""

        result_item = {
            "data_key": data_key
        }
        result.append(path)

len(result)

# List all files
# [manifest_item['key'] + result_files_item['path']
#  for manifest_item in manifest
#  for result_files_item in manifest_item['result_files']]

Skipped entry pdc2023


55

In [11]:
result

['data/photo_copier/event-log.xes',
 'data/synthetic_with_performance_characteristics/event_log.xes',
 'data/synthetic_with_performance_characteristics/event_log_lifecycle_moves.xes',
 'data/env_permit_application_process/event_log.xes',
 'data/job_shop_scheduling/filtered_files/411.xes',
 'data/job_shop_scheduling/filtered_files/412.xes',
 'data/job_shop_scheduling/filtered_files/413.xes',
 'data/job_shop_scheduling/filtered_files/421.xes',
 'data/job_shop_scheduling/filtered_files/422.xes',
 'data/job_shop_scheduling/filtered_files/423.xes',
 'data/job_shop_scheduling/filtered_files/431.xes',
 'data/job_shop_scheduling/filtered_files/432.xes',
 'data/job_shop_scheduling/filtered_files/433.xes',
 'data/job_shop_scheduling/filtered_files/511.xes',
 'data/job_shop_scheduling/filtered_files/512.xes',
 'data/job_shop_scheduling/filtered_files/513.xes',
 'data/job_shop_scheduling/filtered_files/521.xes',
 'data/job_shop_scheduling/filtered_files/522.xes',
 'data/job_shop_scheduling/filtere

In [4]:
class CustomJsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            if math.isnan(obj):
                return None
            else:
                return float(obj)
        if isinstance(obj, np.float64):
            if math.isnan(obj):
                return None
            else:
                return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, pd.Timestamp):
            return str(obj)
        if isinstance(obj, pd.Timedelta):
            return str(obj)
        if isinstance(obj, float) and math.isnan(obj):
            return None
        return super(CustomJsonEncoder, self).default(obj)

In [ ]:
%autoreload
from maxes.data.manifest import manifest
from maxes.analyze_xes import AnalyzeXes

xes_analysis_directory_path = maxes.notebooks.utils.get_project_path('output/xes_analysis')

items_to_analyze = [
    "bpic2020",
    "ccc19",
    "software_data_analytics",
    "bpic2018",
    "daily_living_activities",
    "apache_commons_crypto",
    "hospital_billing",
    "junit",
    "nasa",
    "hospital",
    "road_traffic_fine_management",
    "statechart_workbench",
    "bpic2012",
    "bpic2013",
    "bpic2015",
    "bpic2017",
]

for manifest_item in manifest:
    data_key = manifest_item['key']

    if data_key not in items_to_analyze:
        continue

    if 'result_files' not in manifest_item:
        print(f"Skipped entry {manifest_item['key']}")
        continue


    source_data_directory_path = maxes.notebooks.utils.get_data_path(data_key)

    # Create directory
    result_data_directory_path = os.path.join(xes_analysis_directory_path, data_key)
    Path(result_data_directory_path).mkdir(parents=True, exist_ok=True)

    for result_file in manifest_item['result_files']:
        result_file_key = result_file['key']
        result_file_path = result_file['path']
        result_file_extension = result_file_path.split('.')[-1]

        if result_file_extension != 'xes':
            continue

        source_result_file_path = os.path.join(source_data_directory_path, result_file_path)
        file_name = f"{result_file_key}.json"
        analysis_result_path = os.path.join(result_data_directory_path, file_name)

        analysis = None
        with action_logging(f"Analyzing, data: {data_key}, result_file: {result_file_key}"):
            analyzer = AnalyzeXes()
            analysis = analyzer.analyze(source_result_file_path)

        with action_logging(f"Writing to file: {analysis_result_path}"):
            analysis_json = json.dumps(analysis, ensure_ascii=False, indent=2, cls=CustomJsonEncoder)
            with open(analysis_result_path, mode="w") as f:
                f.write(analysis_json)


In [33]:
pd.Series([1]).describe()['std'].__class__

numpy.float64

In [ ]:
analysis

In [ ]:


data = analyzer.info["Events"]["Count per trace"]
print(json.dumps(data, ensure_ascii=False, indent=4, cls=CustomJsonEncoder, ignore_nan=False))

# Analysis visualization

In [39]:
data = []
for manifest_item in manifest:
    data_key = manifest_item['key']

    result_data_directory_path = os.path.join(xes_analysis_directory_path, data_key)

    for result_file in manifest_item['result_files']:
        result_file_key = result_file['key']
        result_file_path = result_file['path']
        result_file_extension = result_file_path.split('.')[-1]

        if result_file_extension != 'xes':
            continue

        file_name = f"{result_file_key}.json"
        analysis_result_path = os.path.join(result_data_directory_path, file_name)

        with open(analysis_result_path, mode='r') as f:
            analysis = json.load(f)

        data_item = {}
        data.append(data_item)

        data_item['data_key'] = data_key
        data_item['file_key'] = result_file_key
        data_item['size'] = analysis['File']['Size in bytes']
        data_item['size human-readable'] = analysis['File']['Size human-readable']
        data_item['traces_count'] = analysis['Traces']['Count']

        data_item['events_count'] = analysis['Events']['Count in log']
        data_item['events_count_per_trace_min'] = analysis['Events']['Count per trace']['min']
        data_item['events_count_per_trace_max'] = analysis['Events']['Count per trace']['max']
        data_item['events_count_per_trace_mean'] = analysis['Events']['Count per trace']['mean']
        data_item['events_count_per_trace_std'] = analysis['Events']['Count per trace']['std']

        data_item['attributes_count'] = analysis['Events']['Attributes']['Unique count']

        analysis_events_attributes = analysis['Events']['Attributes']['Each']
        data_item['concept_name_unique_values_count'] = analysis_events_attributes['concept:name']['Unique values count']

        if 'lifecycle:transition' in analysis_events_attributes:
            data_item['lifecycle_transition_unique_values_count'] = analysis_events_attributes['lifecycle:transition']['Unique values count']
        else:
            data_item['lifecycle_transition_unique_values_count'] = None




# source, name, size in MB, n of traces, n of events (mean, std)

df = pd.DataFrame(data)
df

,data_key,file_key,size,size human-readable,traces_count,events_count,events_count_per_trace_min,events_count_per_trace_max,events_count_per_trace_mean,events_count_per_trace_std,attributes_count,concept_name_unique_values_count,lifecycle_transition_unique_values_count
0,photo_copier,event_log,10283021,9.8 MiB,100,40995,43.0,2140.0,409.950000,421.930517,4,68,2.0
1,synthetic_with_performance_characteristics,event_log,16565731,15.8 MiB,1,31255,31255.0,31255.0,31255.000000,NaN,17,1,1.0
2,synthetic_with_performance_characteristics,event_log_lifecycle_moves,26094320,24.9 MiB,1,31255,31255.0,31255.0,31255.000000,NaN,18,1,1.0
3,pdc2023,pdc2023_000000,1419653,1.4 MiB,1000,20752,7.0,95.0,20.752000,9.109747,1,28,NaN
4,pdc2023,pdc2023_000001,1347001,1.3 MiB,1000,19619,6.0,85.0,19.619000,9.054824,1,24,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,bpic2015,municipality2,34423078,32.8 MiB,832,44354,1.0,132.0,53.310096,19.906943,13,410,1.0
173,bpic2015,municipality3,46768886,44.6 MiB,1409,59681,3.0,124.0,42.356991,16.144136,13,383,1.0
174,bpic2015,municipality4,36998010,35.3 MiB,1053,47293,1.0,116.0,44.912631,14.954747,13,356,1.0
175,bpic2015,municipality5,46039768,43.9 MiB,1156,59083,5.0,154.0,51.109862,16.036891,13,389,1.0


In [41]:
df[(df['data_key']!= 'pdc2023') & (df['data_key'] != 'job_shop_scheduling')]

,data_key,file_key,size,size human-readable,traces_count,events_count,events_count_per_trace_min,events_count_per_trace_max,events_count_per_trace_mean,events_count_per_trace_std,attributes_count,concept_name_unique_values_count,lifecycle_transition_unique_values_count
0,photo_copier,event_log,10283021,9.8 MiB,100,40995,43.0,2140.0,409.950000,421.930517,4,68,2.0
1,synthetic_with_performance_characteristics,event_log,16565731,15.8 MiB,1,31255,31255.0,31255.0,31255.000000,NaN,17,1,1.0
2,synthetic_with_performance_characteristics,event_log_lifecycle_moves,26094320,24.9 MiB,1,31255,31255.0,31255.0,31255.000000,NaN,18,1,1.0
99,env_permit_application_process,data,4064719,3.9 MiB,1434,8577,1.0,25.0,5.981172,2.166885,6,27,1.0
127,bpic2020,request_for_payment,15185594,14.5 MiB,6886,36796,1.0,20.0,5.343596,1.467491,5,19,NaN
128,bpic2020,domestic_declarations,20497405,19.5 MiB,10500,56437,1.0,24.0,5.374952,1.486415,5,17,NaN
129,bpic2020,international_declarations,29200127,27.8 MiB,6449,72151,3.0,27.0,11.187936,2.740883,5,34,NaN
130,bpic2020,permit_log,33249860,31.7 MiB,7065,86581,3.0,90.0,12.254919,5.585228,5,51,NaN
131,bpic2020,prepaid_travel_cost,7848938,7.5 MiB,2099,18246,1.0,21.0,8.692711,2.254410,5,29,NaN
132,ccc19,data,806604,787.7 KiB,20,1394,52.0,118.0,69.700000,14.857569,11,29,2.0


In [45]:
%autoreload
from maxes.data.manifest_reader import generate_data_files_file

generate_data_files_file()

photo_copier
synthetic_with_performance_characteristics
pdc2023
env_permit_application_process
job_shop_scheduling
lawsuits_brazil
bpic2020
ccc19
software_data_analytics
bpic2018
daily_living_activities
apache_commons_crypto
hospital_billing
junit
nasa
hospital
road_traffic_fine_management
statechart_workbench
bpic2012
bpic2013
bpic2015
bpic2017
